In [1]:
from google.colab import drive

In [2]:
import os

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/My Drive/CH Counts Data'

In [5]:

#Lists all files up to number specified in the google drive folder '/content/drive/My Drive/CH Counts Data'
train_file_numbers = range(1, 2000)
train_file_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in train_file_numbers]

train_file_numbers = range(1, 8329)
train_file_all_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in train_file_numbers]



In [ ]:
import pandas as pd
import random

unique_rows = []
seen_classes = set()

names = random.sample(train_file_all_names, 8329)

#Grabs one row very every unique class seen, is needed for warm start RF
for name in names:
      df = pd.read_csv(name)
      df = df.fillna(0)
      unique_values_in_df = set(df['Mineral Phase'].unique()) - seen_classes
      seen_classes.update(unique_values_in_df)
      for unique_value in unique_values_in_df:
          filtered_df = df[df['Mineral Phase'] == unique_value]
          if not filtered_df.empty:
              unique_row = filtered_df.iloc[0]
              unique_rows.append(unique_row)

#Each unique classes row is added to a df
unique_df = pd.DataFrame(unique_rows)

In [ ]:
len(unique_df)

50

In [ ]:
print(unique_df["Mineral Phase"])

20           Fe-Silicate
1085    AlFe-Silicate_Zn
2167      Fe-Sulphide_Zn
4492             Unknown
6986              Rutile
0               Fe-Oxide
339        AlFe-Silicate
45          AlK-Silicate
345             Zn-Oxide
1166              Quartz
73              Ilmenite
8             Background
1086            Xenotime
3502            Mn-Oxide
5078           Leucoxene
162               Barite
416        AlMn-Silicate
724             Monazite
2201           Celestine
262               Zircon
6619           Amphibole
7386         Al-Silicate
6898             Epidote
2753             Apatite
3665            Pb-Oxide
3132      Fe-sulphide_Pb
3969            Cu-Oxide
6748            Titanite
6807            Al-Oxide
197          Fe-Sulphide
5207       AlCa-Silicate
1787        REE-Silicate
3862             Thorite
493              RM ONLY
2148           Carbonate
2853    Nb-bearing phase
5759         Cassiterite
4384            Chromite
5054                   0
4630        Columbite_Mn


In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Warm start RF model, paramater tuning needed
forest_classifier = RandomForestClassifier(
    warm_start = True,
    n_estimators=1,
    max_depth=None,
    bootstrap=True,
    class_weight="balanced"
)

count = 0


In [ ]:
from sklearn.metrics import accuracy_score
from joblib import load, dump
import warnings
warnings.filterwarnings('ignore')
path_to_save = '/content/drive/MyDrive/forest_classifier.pkl'


test_file_name = "/content/drive/My Drive/CH Counts Data/Group8_2001.csv"

test_df = pd.read_csv(test_file_name)
test_df = test_df.drop(test_df.columns[:126], axis=1)
test_df = pd.concat([test_df,unique_df], ignore_index = True)
test_df = test_df.fillna(0)

X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1].astype(str)


#Adding each file as unique estimator or tree, needs to see all classes first
for  file_name in train_file_names:
    count += 1
    if count%50 == 0:
      dump(forest_classifier,path_to_save)
      y_pred = forest_classifier.predict(X_test)
      accuracy = accuracy_score(y_test, y_pred)
      print(f"Accuracy: {accuracy * 100:.2f}%")
    df = pd.read_csv(file_name)
    df = df.drop(df.iloc[:, 4000:(n_col-2)], axis=1)
    df = df.drop(df.iloc[:, :123], axis=1)
    df = pd.concat([unique_df,df], ignore_index = True)
    df = df.fillna(0)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1].astype(str)
    forest_classifier.fit(X, y)
    forest_classifier.n_estimators += 1




Accuracy: 82.87%
Accuracy: 89.12%
Accuracy: 89.61%
Accuracy: 89.64%
Accuracy: 89.25%
Accuracy: 88.43%
Accuracy: 88.26%
Accuracy: 88.43%
Accuracy: 88.54%
Accuracy: 88.61%
Accuracy: 88.54%
Accuracy: 88.57%
Accuracy: 88.57%
Accuracy: 88.62%
Accuracy: 88.59%
Accuracy: 88.61%
Accuracy: 88.75%
Accuracy: 88.85%
Accuracy: 88.86%
Accuracy: 88.92%
Accuracy: 88.95%
Accuracy: 88.97%
Accuracy: 89.00%
Accuracy: 89.01%
Accuracy: 89.02%
Accuracy: 89.11%
Accuracy: 89.07%
Accuracy: 89.07%
Accuracy: 89.08%
Accuracy: 89.03%


OSError: ignored

In [13]:
from joblib import load, dump
forest_classifier = load("/content/drive/My Drive/forest_classifier.pkl")

In [18]:
import pandas as pd
#Classification Report

#Test file
test_file_name = "/content/drive/My Drive/CH Counts Data/Group8_7000.csv"
df = pd.read_csv(test_file_name)
df = df.fillna(0)
df["Mineral Phase"] = df["Mineral Phase"].astype(str)
n_col = len(df.columns)
df = df.drop(df.iloc[:, 2800:(n_col-2)], axis=1)
df = df.drop(df.iloc[:, :126], axis=1)
X_test = df.iloc[:,:-1]
y_test = df.iloc[:,-1].astype(str)


y_pred = forest_classifier.predict(X_test)

from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

               precision    recall  f1-score   support

  Al-Silicate       0.00      0.00      0.00         0
AlCa-Silicate       0.00      0.00      0.00         4
AlFe-Silicate       0.91      0.41      0.56      1466
 AlK-Silicate       0.67      0.36      0.47        11
AlMn-Silicate       0.62      0.66      0.64       134
    Amphibole       0.76      0.76      0.76       810
      Apatite       1.00      0.20      0.33         5
   Background       0.84      1.00      0.91      1266
      Epidote       0.77      0.55      0.64       287
     Fe-Oxide       0.61      0.90      0.72      1723
  Fe-Silicate       0.67      0.46      0.55       354
     Ilmenite       0.91      0.95      0.93      1182
       Quartz       0.82      0.45      0.58        31
     Titanite       0.87      0.72      0.79       129
   Tourmaline       0.00      0.00      0.00         1
      Unknown       0.00      0.00      0.00        77

     accuracy                           0.76      7480
    macr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
import pandas as pd
from joblib import load, dump
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#Finding out the average accuracy over files specified
validation_file_numbers = range(2010, 2020)
validation_file_all_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in validation_file_numbers]

accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

count = 0
for test_file in validation_file_all_names:
  print(count)
  count +=1
  test_df = pd.read_csv(test_file)
  test_df = test_df.drop(test_df.columns[:126], axis=1)
  test_df = test_df.fillna(0)
  X_test = test_df.iloc[:,:-1]
  y_test = test_df.iloc[:,-1].astype(str)
  y_pred = forest_classifier.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted', zero_division = 0)
  recall = recall_score(y_test, y_pred, average='weighted', zero_division = 0)
  f1 = f1_score(y_test, y_pred, average='weighted',zero_division = 0)
  accuracy_list.append(accuracy)
  precision_list.append(precision)
  recall_list.append(recall)
  f1_list.append(f1)

avg_accuracy = sum(accuracy_list) / len(accuracy_list)
avg_precision = sum(precision_list) / len(precision_list)
avg_recall = sum(recall_list) / len(recall_list)
avg_f1 = sum(f1_list) / len(f1_list)

In [ ]:
print(avg_accuracy,avg_precision,avg_recall,avg_f1)

0.9123588077186223 0.906553199808639 0.9123588077186223 0.8833806366863826


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Old RF trained on first 20 datasets
test_file_name = "/content/drive/My Drive/CH Counts Data/Group8_0026.csv"
test_df = pd.read_csv(test_file_name)
test_df = test_df.drop(test_df.columns[:126], axis=1)

test_df = pd.concat([test_df,unique_df], ignore_index = True)
test_df = test_df.fillna(0)



X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1].astype(str)
print(len(y_test.unique()))


y_pred = forest_classifier.predict(X_test)


from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")


31
Accuracy: 91.68%


In [ ]:
for index, estimator in enumerate(forest_classifier.estimators_):
    y_pred_train_tree = estimator.predict(X)
    unique_y_pred_train_tree = set(y_pred_train_tree)
    if len(unique_y_pred_train_tree) != 49:
        print(f"Tree {index} recognizes {len(unique_y_pred_train_tree)} classes.")

In [ ]:
master_class_list = [
    "0",  # Fill value
    "Fe-Oxide",
    "Background",
    "Ilmenite",
    "Epidote",
    "Amphibole",
    "AlFe-Silicate",
    "Fe-Silicate",
    "Titanite",
    "AlMn-Silicate",
    "Rutile",
    "Zircon",
    "Monazite",
    "Quartz",
    "Cassiterite",
    "AlK-Silicate",
    "Barite",
    "Al-Silicate",
    "Chromite",
    "Unknown",
    "Leucoxene",
    "Fe-Sulphide",
    "AlCa-Silicate",
    "Apatite",
    "RM ONLY",
    "Xenotime",
    "Carbonate",
    "Thorite",
    "AlFe-Silicate_Zn",
    "Al-Oxide",
    "Fe-Sulphide_Cu",
    "W-Oxide",
    "Zn-Oxide",
    "Fe-sulphide_Pb",
    "Celestine",
    "Mn-Oxide",
    "Columbite_Mn",
    "Cu-Oxide",
    "Nb-bearing phase",
    "REE-Silicate",
    "Molybdenite",
    "Tourmaline",
    "Cinnabar",
    "MgAl-Oxide",
    "Fe-Sulphide_Zn",
    "Bi-bearing phase",
    "Fe-Sulphide_As",
    "Pb-Oxide",
    "Tapiolite (TaNb)",
    "Perovskite",
    "V-bearing phase",
    "Fe-Sulphide_Ni"
]

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
label_encode = LabelEncoder()
xgb_model = None


# Fit the label encoder with the master class list

params = {
    'objective': 'multi:softmax',  # Objective for multi-class classification
    'eta': 1,   # Learning rate
    'max_depth': 6
}

label_encode.fit(master_class_list)

# Set the number of classes for XGBoost
params['num_class'] = len(master_class_list)

for i, name in enumerate(train_file_names):
    df = pd.read_csv(name)
    n_col = len(df.columns)

    # Modify these lines if you want to drop columns based on the current df
    df = df.drop(df.iloc[:, 2800:(n_col-2)], axis=1)
    df = df.drop(df.iloc[:, :126], axis=1)

    X = df.iloc[:,:-1]
    y = df.iloc[:,-1].astype(str)
    feature_names = df.columns.tolist()

    y = label_encode.transform(y)  # Transform labels using the label encoder

    dtrain = xgb.DMatrix(X, label=y, feature_names=feature_names)

    xgb_model = xgb.train(params, dtrain, xgb_model=xgb_model)
    print(f"Batch {i+1} training completed.")
    xgb_model.save_model("xgboost_model1.model")



ValueError: ignored

In [ ]:
train_file_numbers = range(80, 100)
train_file_all_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in train_file_numbers]
train_file_names



for name in validation_file_name:
  df = pd.read_csv(name)
  df = df.drop(df.columns[:126], axis = 1)
  X_val = df.iloc[:,:-1]
  y_val = df.iloc[:,-1]
  dval = xgb.Dmatrix(X_val,label =y_val)



y_pred = xgb_model.predict(dval)

accuracy = accuracy_score(y_val,y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred.round())


TypeError: ignored

In [ ]:
unique_df.head()

,CH_124,CH_125,CH_126,CH_127,CH_128,CH_129,CH_130,CH_131,CH_132,CH_133,...,CH_4089,CH_4090,CH_4091,CH_4092,CH_4093,CH_4094,CH_4095,CH_4096,Sum,Mineral Phase
20,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.0,9072,Fe-Silicate
2167,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,1,1,1,1.0,3883,Fe-Sulphide_Zn
1085,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,1,1,1.0,4704,AlFe-Silicate_Zn
73,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0.0,8369,Ilmenite
1166,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,1.0,6478,Quartz
